In [16]:
from keras import backend as K
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Activation
from keras.layers import Reshape, Lambda
from keras.layers.merge import add, concatenate
from keras.models import Model
from keras.layers.recurrent import GRU
from keras.models import load_model
from keras.preprocessing.image import img_to_array
import cv2
import numpy as np

In [2]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    y_pred = y_pred[:, :, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [3]:
def get_model(training,img_w,img_h,depth,alphabetLength,absolute_max_string_len):
    conv_filters = 16
    kernel_size = (3, 3)
    pool_size = 2
    time_dense_size = 32
    rnn_size = 512

    act='relu'
    if K.image_data_format()=='channels_first':
      input_shape=(depth,img_w,img_h)
    else:
      input_shape=(img_w,img_h,depth)

    input_data=Input(name='the_input',shape=input_shape,dtype='float32')
    inner =Conv2D(conv_filters,kernel_size,padding='same',activation=act,
              kernel_initializer='he_normal',name='conv1')(input_data)

    inner =MaxPooling2D(pool_size=(pool_size,pool_size),name='max1')(inner)
    inner =Conv2D(conv_filters,kernel_size,padding='same',activation=act,
              kernel_initializer='he_normal',name='conv2')(inner)

    inner =MaxPooling2D(pool_size=(pool_size,pool_size),name='max2')(inner)
    conv_to_rnn_dims = (img_w // (pool_size**2),(img_h // (pool_size**2 )) * conv_filters)
    inner = Reshape(target_shape=conv_to_rnn_dims, name='reshape')(inner)
    inner = Dense(time_dense_size, activation=act, name='dense1')(inner)       

    gru_1 = GRU(rnn_size, return_sequences=True,
          kernel_initializer='he_normal', name='gru1')(inner)
    gru_1b = GRU(rnn_size, return_sequences=True,
           go_backwards=True, kernel_initializer='he_normal',
           name='gru1_b')(inner)
    gru1_merged = add([gru_1, gru_1b])

    gru_2 = GRU(rnn_size, return_sequences=True,
          kernel_initializer='he_normal', name='gru2')(gru1_merged)
    gru_2b = GRU(rnn_size, return_sequences=True, go_backwards=True,
           kernel_initializer='he_normal', name='gru2_b')(gru1_merged)

    # transforms RNN output to character activations:
    inner = Dense(alphabetLength, kernel_initializer='he_normal',name='dense2')(concatenate([gru_2, gru_2b]))
    y_pred = Activation('softmax', name='softmax')(inner)

    labels = Input(name='the_labels', shape=[absolute_max_string_len], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')

    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])
    
    if training:
        return Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)
    else:
        return Model(inputs=[input_data], outputs=y_pred)

In [4]:
model = get_model(False,128,64,3,67,16)

In [7]:
model.load_weights('ocr_best_weights.hdf5')

In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 128, 64, 3)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  448         the_input[0][0]                  
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
__________________________________________________________________________________________________
max2 (MaxP

In [18]:
# character classes 
alphabet = u'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 '
def labels_to_text(labels):
    ret = []
    for c in labels:
        if c == len(alphabet):  # CTC Blank
            ret.append("")
        else:
            ret.append(alphabet[c])
    return "".join(ret)
def decode_predict_ctc(out, top_paths = 1):
    results = []
    beam_width = 5
    if beam_width < top_paths:
      beam_width = top_paths
    for i in range(top_paths):
      lables = K.get_value(K.ctc_decode(out, input_length=np.ones(out.shape[0])*out.shape[1],
                           greedy=False, beam_width=beam_width, top_paths=top_paths)[0][i])[0]
      text = labels_to_text(lables)
      results.append(text)
    return results

def test(modelPath,testPicTest):
    img=cv2.imread(testPicTest)
    img=cv2.resize(img,(128,64))
    img=img_to_array(img)
    img=np.array(img,dtype='float')/255.0
    img=np.expand_dims(img, axis=0)
    img=img.swapaxes(1,2)   
    
    net_out_value = modelPath.predict(img)
    top_pred_texts = decode_predict_ctc(net_out_value)
    return top_pred_texts

In [25]:
print(test(model,'cheats.png'))

[u'taer']
